---

**Series** : [Sudoku](README.md) | **Precedent** : [Sudoku-11-Comparison](Sudoku-11-Comparison.ipynb) | **Suivant** : [Sudoku-13-BDD](Sudoku-13-BDD.ipynb)

# Sudoku-12 : Theorie des Automates Symboliques et Compilation vers SMT

Dans ce notebook, nous explorons la theorie des **automates symboliques** et comment elle peut se **compiler** vers un solveur SMT comme Z3. Nous verrons les concepts de predicats symboliques, d'operations d'automates, et les appliquerons a la resolution de Sudoku.

## Qu'est-ce qu'un Automate Symbolique?

Un **automate symbolique** est une generalisation des automates finis ou les transitions ne sont pas sur des caracteres concrets mais sur des **predicats logiques**.

| Type de transition | Automate classique | Automate symbolique |
|--------------------|-------------------|---------------------|
| **Etiquette** | Caractere concret : 'a', 'b', '1' | Predicat : x >= 1 AND x <= 9 |
| **Alphabet** | Fini et explicite | Infini ou implicite |
| **Exemple** | DFA avec 91 etats pour [10,100] | 1 transition avec predicat |

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :

1. **Comprendre** la theorie des automates symboliques (Symbolic Finite Automata - SFA)
2. **Implementer** des predicats symboliques avec CharSet en C#
3. **Decouvrir** les operations d'automates (union, intersection, complement, produit)
4. **Comprendre** la compilation de contraintes d'automates vers un solveur SMT
5. **Apprecier** les limites de l'approche pure "automates" pour Sudoku

### Prerequis

- [Sudoku-0-Environment](Sudoku-0-Environment.ipynb) - Classes de base Sudoku
- [Sudoku-4-Z3](Sudoku-4-Z3.ipynb) - Bases de Z3 SMT (recommande)
- Connaissance de base de la theorie des automates (DFA/NFA)

### Duree estimee : 1h30

## References

- **Marcus Veanes** - Principal researcher chez Microsoft, pionnier des automates symboliques
- [Automata.Net Repository](https://github.com/AutomataDotNet/Automata) - Bibliotheque C# pour automates symboliques
- [Search-12-SymbolicAutomata](../Search/Foundations/Search-12-SymbolicAutomata.ipynb) - Theorie approfondie

## Note sur l'approche

> **⚠️ Important** : Ce notebook presente la theorie des automates symboliques et montre comment les concepts (predicats, intersection, produit) se compilent naturellement vers des contraintes SMT. Pour une implementation "pure" automates avec BDD/MDD, voir [Sudoku-13](Sudoku-13-BDD.ipynb).

---

## 1. Introduction - Automates Finis vs Symboliques (15 min)

### 1.1 Rappel : Automate Fini Deterministe (DFA)

Un **DFA** est un 5-tuple $(Q, \Sigma, \delta, q_0, F)$ ou :

- $Q$ : ensemble fini d'etats
- $\Sigma$ : alphabet fini (symboles d'entree)
- $\delta : Q \times \Sigma \to Q$ : fonction de transition
- $q_0 \in Q$ : etat initial
- $F \subseteq Q$ : etats finaux (acceptants)

**Exemple** : DFA qui accepte les entiers pairs entre 10 et 100

```
Avec alphabet fini {10, 11, 12, ..., 100} :
- 91 etats pour representer chaque valeur paire
- 91 transitions (une par valeur acceptee)
```

### 1.2 Automate Symbolique (SFA)

Un **SFA** remplace l'alphabet fini par des predicats :

- $\Phi$ : ensemble de predicats logiques sur un alphabet potentiellement infini
- $\delta : Q \times \Phi \to Q$ : transitions symboliques

**Meme exemple** : SFA pour entiers pairs entre 10 et 100

```
Alphabet : tous les entiers (infini)
- 2 etats (initial, final)
- 1 transition avec predicat : x >= 10 AND x <= 100 AND x % 2 == 0
```

### 1.3 Metaphore vs Implementation

> **⚠️ Hônneteté intellectuelle** : Il existe un "fossé" entre la theorie des automates symboliques et leur implementation pratique pour des problemes comme Sudoku.

| Aspect | Theorie automata | Implementation pratique (Z3) |
|--------|-----------------|------------------------------|
| **Modelisation** | Etats + transitions symboliques | Variables + contraintes SMT |
| **Produit d'automates** | Construction explicite $A \times B$ | Conjonction de contraintes |
| **Acceptation** | Parcours d'automate | SAT solving |
| **Avantage** | Theorie bien etablie | Performance optimale |

**Ce que nous ferons dans ce notebook** :
1. Sections 2-3 : Vrais automates symboliques avec CharSet (theorie pure)
2. Sections 4-5 : Compilation vers Z3 (pragmatisme)
3. Section 6 : Discussion sur quand utiliser chaque approche

Cette dualite theorie/pratique est exactement ce que fait la bibliotheque **Automata.Net** : elle fournit des operations d'automates de haut niveau qui sont compilees vers des solveurs (BDD, Z3) pour l'execution.

---

## 2. Predicats Symboliques - CharSet (20 min)

Pour construire des automates symboliques, nous avons besoin de represente des **predicats** sur des ensembles de caracteres. Commencons par implementer une classe `CharSet` qui represente des predicats sur des caracteres.

### 2.1 Classe CharSet - Predicats sur les Caracteres

La classe `CharSet` permet de representer :
- Un caractere unique : `CharSet.Single('5')`
- Une plage de caracteres : `CharSet.Range('1', '9')`
- Une union de predicats : `digit.Or(letter)`
- Une intersection : `lowercase.And(vowel)`
- Un complement : `notDigit = digit.Not()`

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;

/// <summary>
/// Represente un predicat symbolique sur des caracteres.
/// Un CharSet peut etre :
/// - Un caractere unique
/// - Une plage de caracteres
/// - Une union/intersection de predicats
/// - Le complement d'un predicat
/// </summary>
public abstract class CharSet
{
    // Singleton pour l'ensemble vide
    private static readonly CharSetEmpty _empty = new CharSetEmpty();
    public static CharSet Empty => _empty;
    
    // Singleton pour l'univers (tous les caracteres)
    private static readonly CharSetAll _all = new CharSetAll();
    public static CharSet All => _all;
    
    /// <summary>
    /// Cree un CharSet pour un caractere unique.
    /// </summary>
    public static CharSet Single(char c) => new CharSetSingle(c);
    
    /// <summary>
    /// Cree un CharSet pour une plage de caracteres [min, max].
    /// </summary>
    public static CharSet Range(char min, char max) => new CharSetRange(min, max);
    
    /// <summary>
    /// Cree un CharSet pour un ensemble de caracteres.
    /// </summary>
    public static CharSet Set(params char[] chars) => new CharSetSet(chars);
    
    /// <summary>
    /// Teste si un caractere satisfait le predicat.
    /// </summary>
    public abstract bool Contains(char c);
    
    /// <summary>
    /// Union logique (OR) de deux predicats.
    /// </summary>
    public CharSet Or(CharSet other) => new CharSetUnion(this, other);
    
    /// <summary>
    /// Intersection logique (AND) de deux predicats.
    /// </summary>
    public CharSet And(CharSet other) => new CharSetIntersection(this, other);
    
    /// <summary>
    /// Complement logique (NOT) du predicat.
    /// </summary>
    public CharSet Not() => new CharSetComplement(this);
    
    /// <summary>
    /// Retourne une representation lisible du predicat.
    /// </summary>
    public abstract string ToPredicateString();
    
    // Classes internes pour chaque type de predicat
    
    private sealed class CharSetEmpty : CharSet
    {
        public override bool Contains(char c) => false;
        public override string ToPredicateString() => "false";
    }
    
    private sealed class CharSetAll : CharSet
    {
        public override bool Contains(char c) => true;
        public override string ToPredicateString() => "true";
    }
    
    private sealed class CharSetSingle : CharSet
    {
        private readonly char _value;
        public CharSetSingle(char value) => _value = value;
        public override bool Contains(char c) => c == _value;
        public override string ToPredicateString() => $"x == '{_value}'";
    }
    
    private sealed class CharSetRange : CharSet
    {
        private readonly char _min;
        private readonly char _max;
        public CharSetRange(char min, char max)
        {
            _min = min;
            _max = max;
        }
        public override bool Contains(char c) => c >= _min && c <= _max;
        public override string ToPredicateString() => $"x >= '{_min}' AND x <= '{_max}'";
    }
    
    private sealed class CharSetSet : CharSet
    {
        private readonly char[] _chars;
        public CharSetSet(char[] chars) => _chars = chars;
        public override bool Contains(char c) => _chars.Contains(c);
        public override string ToPredicateString() => $"x IN {{{string.Join(", ", _chars.Select(c => $"'{c}'"))}}}";
    }
    
    private sealed class CharSetUnion : CharSet
    {
        private readonly CharSet _left;
        private readonly CharSet _right;
        public CharSetUnion(CharSet left, CharSet right)
        {
            _left = left;
            _right = right;
        }
        public override bool Contains(char c) => _left.Contains(c) || _right.Contains(c);
        public override string ToPredicateString() => $"({_left.ToPredicateString()}) OR ({_right.ToPredicateString()})";
    }
    
    private sealed class CharSetIntersection : CharSet
    {
        private readonly CharSet _left;
        private readonly CharSet _right;
        public CharSetIntersection(CharSet left, CharSet right)
        {
            _left = left;
            _right = right;
        }
        public override bool Contains(char c) => _left.Contains(c) && _right.Contains(c);
        public override string ToPredicateString() => $"({_left.ToPredicateString()}) AND ({_right.ToPredicateString()})";
    }
    
    private sealed class CharSetComplement : CharSet
    {
        private readonly CharSet _inner;
        public CharSetComplement(CharSet inner) => _inner = inner;
        public override bool Contains(char c) => !_inner.Contains(c);
        public override string ToPredicateString() => $"NOT ({_inner.ToPredicateString()})";
    }
}

Console.WriteLine("Classe CharSet definie avec succes.");
Console.WriteLine("Types de predicats supportes :");
Console.WriteLine("  - Single(char) : Un caractere unique");
Console.WriteLine("  - Range(min, max) : Une plage de caracteres");
Console.WriteLine("  - Set(params[]) : Un ensemble explicite");
Console.WriteLine("  - Or/And/Not : Operations logiques");

Classe CharSet definie avec succes.


Types de predicats supportes :


  - Single(char) : Un caractere unique


  - Range(min, max) : Une plage de caracteres


  - Set(params[]) : Un ensemble explicite


  - Or/And/Not : Operations logiques


### 2.2 Exemples de Predicats CharSet

Testons notre classe `CharSet` avec differents types de predicats.

In [ ]:
// Exemple 1 : Predicat pour chiffres Sudoku (1-9)
CharSet sudokuDigit = CharSet.Range('1', '9');

Console.WriteLine("=== Exemple 1 : Chiffres Sudoku (1-9) ===");
Console.WriteLine($"Predicat : {sudokuDigit.ToPredicateString()}");
Console.WriteLine();
Console.WriteLine("Tests d'appartenance :");
foreach (char c in "0123456789X.")
{
    bool inSet = sudokuDigit.Contains(c);
    string status = inSet ? "[OUI]" : "[NON]";
    Console.WriteLine($"  '{c}' : {status}");
}

Console.WriteLine();
Console.WriteLine("=== Exemple 2 : Predicats composes ===");

// Chiffres pairs
CharSet evenDigit = CharSet.Set('2', '4', '6', '8');
Console.WriteLine($"Chiffres pairs : {evenDigit.ToPredicateString()}");

// Chiffres impairs - VRAI complement dans le domaine 1-9
// odd = (domain 1-9) AND (NOT pairs)
CharSet domain = CharSet.Range('1', '9');
CharSet oddDigit = domain.And(evenDigit.Not());
Console.WriteLine($"Chiffres impairs (vrai complement) : {oddDigit.ToPredicateString()}");

// Petits chiffres (1-5)
CharSet smallDigit = CharSet.Range('1', '5');
Console.WriteLine($"Petits chiffres (1-5) : {smallDigit.ToPredicateString()}");

// Union : petits OU pairs
CharSet smallOrEven = smallDigit.Or(evenDigit);
Console.WriteLine($"Petits OU pairs : {smallOrEven.ToPredicateString()}");

Console.WriteLine();
Console.WriteLine("Tests de predicat compose (petits OU pairs) :");
foreach (char c in "123456789")
{
    bool inSet = smallOrEven.Contains(c);
    if (inSet)
        Console.WriteLine($"  '{c}' est accepte");
}

=== Exemple 1 : Chiffres Sudoku (1-9) ===


Predicat : x >= '1' AND x <= '9'


Tests d'appartenance :


  '0' : [NON]


  '1' : [OUI]


...

  '3' est accepte


  '4' est accepte


  '5' est accepte


  '6' est accepte


  '8' est accepte


#### Interpretation : Predicats CharSet

**Sortie obtenue** : La classe `CharSet` permet de creer des predicats logiques sur des caracteres.

**Exemple 1 - Chiffres Sudoku** :
| Caractere | Accepte | Explication |
|-----------|---------|-------------|
| '0' | Non | Hors plage [1-9] |
| '1'-'9' | Oui | Dans plage [1-9] |
| 'X', '.' | Non | Hors plage [1-9] |

**Exemple 2 - Predicats composes** :
- `Range('1', '5')` : Predicat $x \geq 1 \land x \leq 5$
- `Set('2','4','6','8')` : Predicat $x \in \{2,4,6,8\}$
- `smallOrEven` : Predicat $(1 \leq x \leq 5) \lor (x \in \{2,4,6,8\})$

**Points cles** :
1. Les predicats peuvent etre combines avec `Or`, `And`, `Not`
2. La methode `Contains(char)` teste si un caractere satisfait le predicat
3. Les predicats sont representes de manière symbolique (formules, pas enumeration)

> **Note theorique** : Ces predicats sont exactement ce qui distingue les automates symboliques des automates classiques. Dans un DFA classique, chaque transition est etiquettee par un caractere concret. Dans un SFA, chaque transition est etiquettee par un predicat comme `Range('1', '9')`.

---

## 3. Automate Symbolique Simple (20 min)

Maintenant que nous avons des predicats symboliques, nous pouvons construire un **automate symbolique** (SFA). Commençons par un automate simple qui accepte les sequences de 9 chiffres distincts.

### 3.1 Classe SymbolicFiniteAutomaton

Cette classe represente un automate symbolique avec :
- Des etats nommes
- Des transitions etiquettees par des predicats `CharSet`
- Un etat initial et des etats finaux

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

/// <summary>
/// Represente une transition symbolique dans un automate.
/// </summary>
public class SymbolicTransition
{
    public string FromState { get; }
    public string ToState { get; }
    public CharSet Predicate { get; }
    
    public SymbolicTransition(string from, string to, CharSet predicate)
    {
        FromState = from;
        ToState = to;
        Predicate = predicate;
    }
    
    public override string ToString() => $"{FromState} --[{Predicate.ToPredicateString()}]--> {ToState}";
}

/// <summary>
/// Automate symbolique avec predicats CharSet.
/// </summary>
public class SymbolicFiniteAutomaton
{
    private HashSet<string> _states;
    private List<SymbolicTransition> _transitions;
    private string _initialState;
    private HashSet<string> _finalStates;
    
    public string Name { get; }
    public IReadOnlySet<string> States => _states;
    public IReadOnlyList<SymbolicTransition> Transitions => _transitions;
    public string InitialState => _initialState;
    public IReadOnlySet<string> FinalStates => _finalStates;
    
    public SymbolicFiniteAutomaton(string name)
    {
        Name = name;
        _states = new HashSet<string>();
        _transitions = new List<SymbolicTransition>();
        _finalStates = new HashSet<string>();
    }
    
    /// <summary>
    /// Ajoute un etat a l'automate.
    /// </summary>
    public SymbolicFiniteAutomaton AddState(string state, bool isInitial = false, bool isFinal = false)
    {
        _states.Add(state);
        if (isInitial)
        {
            if (_initialState != null)
                throw new ArgumentException($"Etat initial deja defini : {_initialState}");
            _initialState = state;
        }
        if (isFinal)
            _finalStates.Add(state);
        return this;
    }
    
    /// <summary>
    /// Ajoute une transition symbolique.
    /// </summary>
    public SymbolicFiniteAutomaton AddTransition(string from, string to, CharSet predicate)
    {
        if (!_states.Contains(from))
            throw new ArgumentException($"Etat source inconnu : {from}");
        if (!_states.Contains(to))
            throw new ArgumentException($"Etat destination inconnu : {to}");
        
        _transitions.Add(new SymbolicTransition(from, to, predicate));
        return this;
    }
    
    /// <summary>
    /// Teste si l'automate accepte une sequence de caracteres.
    /// </summary>
    public bool Accepts(string input)
    {
        if (_initialState == null)
            throw new InvalidOperationException("Pas d'etat initial defini");
        
        return AcceptsRecursive(_initialState, input, 0);
    }
    
    private bool AcceptsRecursive(string currentState, string input, int position)
    {
        // Cas de base : fin de l'entree
        if (position == input.Length)
            return _finalStates.Contains(currentState);
        
        char currentChar = input[position];
        bool accepted = false;
        
        // Trouver toutes les transitions applicables
        foreach (var transition in _transitions.Where(t => t.FromState == currentState))
        {
            if (transition.Predicate.Contains(currentChar))
            {
                // Transition possible, continuer recursivement
                if (AcceptsRecursive(transition.ToState, input, position + 1))
                    accepted = true;
            }
        }
        
        return accepted;
    }
    
    /// <summary>
    /// Retourne une representation textuelle de l'automate.
    /// </summary>
    public override string ToString()
    {
        var sb = new System.Text.StringBuilder();
        sb.AppendLine($"Automate Symbolique : {Name}");
        sb.AppendLine($"  Etats : {string.Join(", ", _states)}");
        sb.AppendLine($"  Etat initial : {_initialState}");
        sb.AppendLine($"  Etats finaux : {string.Join(", ", _finalStates)}");
        sb.AppendLine("  Transitions :");
        foreach (var t in _transitions)
            sb.AppendLine($"    {t}");
        return sb.ToString();
    }
}

Console.WriteLine("Classe SymbolicFiniteAutomaton definie avec succes.");

Classe SymbolicFiniteAutomaton definie avec succes.


### 3.2 Exemple : Automate pour Chiffres Sudoku

Creons un automate simple qui accepte uniquement des sequences de 9 chiffres entre 1 et 9.

In [ ]:
// Automate qui accepte exactement 9 chiffres Sudoku (1-9)
var digitAutomaton = new SymbolicFiniteAutomaton("SudokuDigitValidator");

// Ajouter les etats : q0 (initial) -> q1 -> ... -> q9 (final)
for (int i = 0; i <= 9; i++)
{
    digitAutomaton.AddState($"q{i}", isInitial: (i == 0), isFinal: (i == 9));
}

// Ajouter les transitions : chaque chiffre Sudoku avance d'un etat
CharSet sudokuDigit = CharSet.Range('1', '9');
for (int i = 0; i < 9; i++)
{
    digitAutomaton.AddTransition($"q{i}", $"q{i + 1}", sudokuDigit);
}

Console.WriteLine(digitAutomaton);

// Tests
Console.WriteLine("\n=== Tests d'acceptation ===");
string[] testCases = {
    "123456789",    // Valide : 9 chiffres
    "111111111",    // Valide : 9 chiffres (meme si repetes)
    "12345678",     // Invalide : 8 chiffres
    "1234567890",   // Invalide : 10 chiffres
    "12345678X",    // Invalide : contient 'X'
    "               ", // Invalide : vide
    "528317946"     // Valide : 9 chiffres
};

foreach (var test in testCases)
{
    bool accepted = digitAutomaton.Accepts(test);
    string display = string.IsNullOrEmpty(test) ? "<vide>" : test;
    string status = accepted ? "[ACCEPTED]" : "[REJECTED]";
    Console.WriteLine($"  {display,12} : {status}");
}

Automate Symbolique : SudokuDigitValidator
  Etats : q0, q1, q2, q3, q4, q5, q6, q7, q8, q9
  Etat initial : q0
  Etats finaux : q9
  Transitions :
    q0 --[x >= '1' AND x <= '9']--> q1
    q1 --[x >= '1' AND x <= '9']--> q2
    q2 --[x >= '1' AND x <= '9']--> q3
    q3 --[x >= '1' AND x <= '9']--> q4
    q4 --[x >= '1' AND x <= '9']--> q5
    q5 --[x >= '1' AND x <= '9']--> q6
    q6 --[x >= '1' AND x <= '9']--> q7
    q7 --[x >= '1' AND x <= '9']--> q8
    q8 --[x >= '1' AND x <= '9']--> q9




=== Tests d'acceptation ===


     123456789 : [ACCEPTED]


     111111111 : [ACCEPTED]


      12345678 : [REJECTED]


    1234567890 : [REJECTED]


     12345678X : [REJECTED]


                  : [REJECTED]


     528317946 : [ACCEPTED]


#### Interpretation : Automate pour Chiffres Sudoku

**Sortie obtenue** : L'automate accepte uniquement les sequences de 9 chiffres dans [1-9].

| Input | Longueur | Contenu | Accepte | Pourquoi |
|-------|----------|---------|---------|----------|
| "123456789" | 9 | Tous 1-9 | Oui | Correct |
| "111111111" | 9 | Tous 1-9 | Oui | Correct (pas de contrainte d'unicite) |
| "12345678" | 8 | Tous 1-9 | Non | Pas assez de caracteres |
| "1234567890" | 10 | Contient '0' | Non | Trop de caracteres + '0' invalide |
| "12345678X" | 9 | Contient 'X' | Non | 'X' pas dans [1-9] |
| "" | 0 | Vide | Non | Pas dans etat final |

**Structure de l'automate** :
```
      [1-9]     [1-9]           [1-9]
q0 ------> q1 ------> q2 ---> ... ---> q9 (final)
```

**Point important** : Cet automate verifie uniquement le **format** (9 chiffres valides), pas la contrainte d'unicite. Pour verifier que tous les chiffres sont distincts, nous aurions besoin d'un automate plus complexe avec $9! = 362880$ etats (une pour chaque permutation). C'est ici que les automates symboliques avec Z3 deviennent plus interessants.

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

/// <summary>
/// Extension operations for symbolic automata - Automata.Net style
/// </summary>
public static class AutomataOperations
{
    /// <summary>
    /// Intersection de deux automates : L(A) and L(B)
    /// Retourne true s'il existe au moins un mot accepte par les deux.
    ///
    /// NOTE IMPORTANTE : Ceci est une version DEMO pédagogique, PAS une procédure
    /// de décision correcte! Une vraie implémentation Automata.Net utiliserait:
    ///   - MkProduct pour construire un automate produit explicite
    ///   - Puis vérifier si l'automate produit accepte au moins un mot
    ///
    /// Ici, on teste simplement quelques mots candidats - suffisant pour la démo,
    /// mais pas correct pour une vraie décision "intersection non-vide".
    /// </summary>
    public static bool IntersectsWith(SymbolicFiniteAutomaton A, SymbolicFiniteAutomaton B)
    {
        // Approche naive : tester tous les mots jusqu'a une certaine longueur
        // Dans Automata.Net, cela utilise MkProduct pour construire un automate produit
        // puis verifie si le langage resultant est non-vide.
        
        // Pour notre demo, on teste quelques mots candidats
        var candidateWords = new[] {
            "123456789", "111111111", "5", "555555555", "12345678", 
            "1234567890", "123467899", "987654321"
        };
        
        foreach (var word in candidateWords)
        {
            if (A.Accepts(word) && B.Accepts(word))
                return true;
        }
        
        return false;
    }
    
    /// <summary>
    /// Union de deux automates : L(A) or L(B)
    /// Retourne true si le mot est accepte par A ou par B.
    /// </summary>
    public static bool AcceptsUnion(SymbolicFiniteAutomaton A, SymbolicFiniteAutomaton B, string word)
    {
        return A.Accepts(word) || B.Accepts(word);
    }
    
    /// <summary>
    /// Complement : trouve un mot non accepte par A
    /// Retourne un mot qui n'est PAS dans L(A).
    /// </summary>
    public static string FindNonMember(SymbolicFiniteAutomaton A, string alphabet)
    {
        // Essayer des mots de plus en plus longs
        for (int len = 1; len <= 10; len++)
        {
            var words = GenerateWords(alphabet, len);
            foreach (var word in words)
            {
                if (!A.Accepts(word))
                    return word;
            }
        }
        return null; // A accepte tout (language universel)
    }
    
    private static IEnumerable<string> GenerateWords(string alphabet, int length)
    {
        if (length == 0)
        {
            yield return "";
            yield break;
        }
        
        // Version simplifiee - pour vrai cas, utiliser GenerateAllCombos
        foreach (var c in alphabet)
        {
            foreach (var tail in GenerateWords(alphabet, length - 1))
            {
                yield return c + tail;
            }
        }
    }
}

Console.WriteLine("=== Operations Automata.Net (Version Démo) ===");
Console.WriteLine("Operations disponibles :");
Console.WriteLine("  - IntersectsWith(A, B) : Verifie si L(A) inter L(B) est non-vide");
Console.WriteLine("  ⚠️  NOTE : Ceci n'est PAS une procédure de décision correcte!");
Console.WriteLine("     Une vraie implémentation construirait un automate produit explicite.");
Console.WriteLine("  - AcceptsUnion(A, B, word) : Verifie si word in L(A) U L(B)");
Console.WriteLine("  - FindNonMember(A, alphabet) : Trouve un mot hors de L(A)");
Console.WriteLine();

=== Operations Automata.Net (Version Démo) ===


Operations disponibles :


  - IntersectsWith(A, B) : Verifie si L(A) inter L(B) est non-vide


  ⚠️  NOTE : Ceci n'est PAS une procédure de décision correcte!


     Une vraie implémentation construirait un automate produit explicite.


  - AcceptsUnion(A, B, word) : Verifie si word in L(A) U L(B)


  - FindNonMember(A, alphabet) : Trouve un mot hors de L(A)


---

## 4. Esprit Automata.Net en 10 minutes (15 min)

Dans cette section, nous decouvrons les operations fondamentales de la bibliotheque **Automata.Net** - operations qui rendent les automates symboliques si puissants. Meme sans utiliser la DLL, nous pouvons comprendre et implementer ces concepts.

### 4.1 Operations Fondamentales sur Automates

Les automates symboliques supportent des operations d'algebre de langages :

| Operation | Symbole | Signification | Exemple Automata.Net |
|-----------|----------|---------------|----------------------|
| **Union** | $L(A) \cup L(B)$ | Strings acceptes par A OU B | `A.Union(B)` |
| **Intersection** | $L(A) \cap L(B)$ | Strings acceptes par A ET B | `A.Intersect(B)` |
| **Complement** | $\Sigma^* \setminus L(A)$ | Strings NON acceptes par A | `A.Complement()` |
| **Produit** | $L(A \times B)$ | Equivalent a l'intersection | `A.Product(B)` |
| **Difference** | $L(A) \setminus L(B)$ | Dans A mais pas dans B | `A.Minus(B)` |

### 4.2 Exemple Concret : Mots de 9 Chiffres

Construisons deux automates pour illustrer ces operations :

**Automate A** : Accepte les chaînes de exactement 9 chiffres (1-9)
**Automate B** : Accepte les chaînes contenant au moins un '5'

```csharp
// Automate A : exactement 9 chiffres Sudoku
var A = new SymbolicFiniteAutomaton("Exactly9Digits");
// ... etats q0...q9, transitions [1-9] ...

// Automate B : contient au moins un '5'
var B = new SymbolicFiniteAutomaton("Contains5");
// ... structure plus complexe avec etat de memoire ...
```

**Operations que nous pouvons effectuer** :

1. **Intersection** $A \cap B$ : Chaînes de 9 chiffres qui contiennent un '5'
2. **Complement** $\neg A$ : Toutes les chaînes SAUF celles de 9 chiffres
3. **Union** $A \cup B$ : Chaînes de 9 chiffres OU contenant un '5'

### 4.3 Implementation des Operations Automata

Etendons notre classe avec les operations fondamentales :

---

## 5. Compilation vers SMT - Sudoku Complet (25 min)

> **⚠️ Approche pragmatique** : Dans cette section, nous compilons les contraintes d'automates symboliques vers un solveur SMT (Z3). C'est l'approche utilisee par Automata.Net en production pour des problemes avec contraintes arithmetiques.

Pour resoudre un Sudoku complet, nous devons combiner les contraintes de :
- 9 lignes (distinctes)
- 9 colonnes (distinctes)
- 9 blocs 3x3 (distincts)

### 5.1 Du Produit d'Automates aux Contraintes SMT

**Theorie** : Le produit d'automates $A_1 \times A_2$ accepte l'intersection $L(A_1) \cap L(A_2)$.

**Pratique (Automata.Net)** : Au lieu de construire explicitement un automate produit avec $|Q_1| \times |Q_2|$ etats, la bibliotheque compile les operations vers :
- **BDD** (Binary Decision Diagrams) pour les predicats de caracteres
- **Z3** pour les contraintes arithmetiques complexes

**Notre approche** : Nous utilisons directement Z3 comme backend, ce qui donne :

| Aspect | Produit explicite | Compilation vers Z3 (notre approche) |
|--------|------------------|----------------------------------|
| **Construction** | $(q_1, q_2) \to (q'_1, q'_2)$ | Conjonction de contraintes `Distinct` |
| **Verification** | Parcours d'automate | SAT solving |
| **Avantage** | Theoriquement elegant | Pratique et performant |

### 5.2 Encodage des Contraintes Sudoku en SMT

Traduisons chaque contrainte Sudoku en formule Z3 :

| Contrainte | Theorie automata | Encodage Z3 |
|-----------|-----------------|-------------|
| **Ligne i** | Automate $A_{row_i}$ | `Distinct(x_i0, ..., x_i8)` |
| **Colonne j** | Automate $A_{col_j}$ | `Distinct(x_0j, ..., x_8j)` |
| **Bloc b** | Automate $A_{block_b}$ | `Distinct` des 9 cellules du bloc |
| **Intersection** | $A_{sudoku} = \bigcap_{k=1}^{27} A_k$ | Conjonction de tous les `Distinct` |

**Note** : L'encodage direct en Z3 elimine le besoin de construire explicitement 27 automates. C'est exactement ce que fait Automata.Net en interne via sa classe `CharSetSolver`.

In [ ]:
// Charger Z3 depuis Sudoku-0 si disponible
#r "nuget:Microsoft.Z3,*"

using Microsoft.Z3;
using System;
using System.Collections.Generic;
using System.Linq;

Console.WriteLine("Z3 charge avec succes.");
Console.WriteLine();
Console.WriteLine("=== Note Importante ===");
Console.WriteLine("Dans cette section, nous utilisons Z3 comme backend pour");
Console.WriteLine("la compilation de contraintes d'automates symboliques.");
Console.WriteLine("C'est l'approche pragmatique adoptee par Automata.Net.");

Installing Packages Microsoft.Z3

Z3 charge avec succes.


=== Note Importante ===


Dans cette section, nous utilisons Z3 comme backend pour


la compilation de contraintes d'automates symboliques.


C'est l'approche pragmatique adoptee par Automata.Net.


### 5.3 Automate Symbolique pour Ligne Sudoku avec Z3

Nous allons creer une classe qui combine :
- La structure d'automate symbolique (etats, transitions)
- La puissance de Z3 pour representer les predicats et verifier la distinctivite

**Note importante** : Cette classe n'est pas un "vrai" automate symbolique avec des etats explicites. C'est une **compilation** vers Z3 qui capture l'essence de la contrainte de ligne (valeurs distinctes).

In [ ]:
/// <summary>
/// Automate symbolique pour une ligne de Sudoku avec verification de distinctivite via Z3.
/// </summary>
public class SudokuRowAutomaton
{
    private Context _ctx;
    private Solver _solver;
    private IntExpr[] _cells;
    private int _rowIndex;
    
    public int RowIndex => _rowIndex;
    
    public SudokuRowAutomaton(int rowIndex)
    {
        _rowIndex = rowIndex;
        _ctx = new Context();
        _solver = _ctx.MkSolver();
        
        // Creer les variables pour les 9 cellules de la ligne
        _cells = new IntExpr[9];
        for (int j = 0; j < 9; j++)
        {
            _cells[j] = _ctx.MkIntConst($"x_{rowIndex}_{j}");
        }
        
        // Ajouter la contrainte de distinctivite
        _solver.Add(_ctx.MkDistinct(_cells));
        
        // Ajouter les contraintes de domaine [1, 9]
        foreach (var cell in _cells)
        {
            _solver.Add(_ctx.MkAnd(
                _ctx.MkGe(cell, _ctx.MkInt(1)),
                _ctx.MkLe(cell, _ctx.MkInt(9))
            ));
        }
    }
    
    /// <summary>
    /// Verifie si une sequence de 9 valeurs est acceptee par cet automate.
    /// </summary>
    public bool Accepts(int[] values)
    {
        if (values.Length != 9)
            throw new ArgumentException("Une ligne Sudoku doit contenir exactement 9 valeurs");
        
        // Creer un solver temporaire pour cette verification
        var testSolver = _ctx.MkSolver();
        
        // Ajouter la contrainte de distinctivite
        testSolver.Add(_ctx.MkDistinct(_cells));
        
        // Ajouter les contraintes de domaine
        foreach (var cell in _cells)
        {
            testSolver.Add(_ctx.MkGe(cell, _ctx.MkInt(1)));
            testSolver.Add(_ctx.MkLe(cell, _ctx.MkInt(9)));
        }
        
        // Ajouter les valeurs specifiques a tester
        for (int j = 0; j < 9; j++)
        {
            testSolver.Add(_ctx.MkEq(_cells[j], _ctx.MkInt(values[j])));
        }
        
        // Verifier la satisfiabilite
        Status result = testSolver.Check();
        return result == Status.SATISFIABLE;
    }
    
    /// <summary>
    /// Retourne une modele (une instance valide) de la ligne.
    /// </summary>
    public int[] GetModel()
    {
        Status status = _solver.Check();
        if (status != Status.SATISFIABLE)
            return null;
        
        Model model = _solver.Model;  // FIX: Model is a property, not a method
        int[] values = new int[9];
        
        for (int j = 0; j < 9; j++)
        {
            Expr eval = model.Evaluate(_cells[j]);
            values[j] = ((IntNum)eval).Int;
        }
        
        return values;
    }
    
    public override string ToString() => $"SudokuRowAutomaton(Row={_rowIndex})";
}

Console.WriteLine("Classe SudokuRowAutomaton definie avec succes.");

Classe SudokuRowAutomaton definie avec succes.


### 5.4 Tests de l'Automate de Ligne

In [ ]:
// Creer un automate pour la ligne 0
var rowAuto = new SudokuRowAutomaton(0);

Console.WriteLine("=== Automate de Ligne Sudoku ===");
Console.WriteLine(rowAuto);
Console.WriteLine();

// Test 1 : Sequence valide (permutation de 1-9)
Console.WriteLine("Test 1 : Sequence valide (permutation de 1-9)");
int[] validRow = {1, 2, 3, 4, 5, 6, 7, 8, 9};
bool result1 = rowAuto.Accepts(validRow);
Console.WriteLine($"  Sequence : [{string.Join(", ", validRow)}]");
Console.WriteLine($"  Acceptee? {result1}");
Console.WriteLine($"  Attendu : True (toutes les valeurs sont distinctes)");
Console.WriteLine();

// Test 2 : Sequence invalide (doublon)
Console.WriteLine("Test 2 : Sequence invalide (doublon de 1)");
int[] invalidRow = {1, 1, 3, 4, 5, 6, 7, 8, 9};
bool result2 = rowAuto.Accepts(invalidRow);
Console.WriteLine($"  Sequence : [{string.Join(", ", invalidRow)}]");
Console.WriteLine($"  Acceptee? {result2}");
Console.WriteLine($"  Attendu : False (la valeur 1 apparait deux fois)");
Console.WriteLine();

// Test 3 : Sequence avec valeur hors domaine
Console.WriteLine("Test 3 : Sequence avec valeur hors domaine");
int[] outOfRangeRow = {1, 2, 3, 4, 5, 6, 7, 8, 10};
bool result3 = rowAuto.Accepts(outOfRangeRow);
Console.WriteLine($"  Sequence : [{string.Join(", ", outOfRangeRow)}]");
Console.WriteLine($"  Acceptee? {result3}");
Console.WriteLine($"  Attendu : False (10 n'est pas dans [1, 9])");
Console.WriteLine();

// Obtenir un modele valide
Console.WriteLine("Generation d'un modele valide :");
int[] model = rowAuto.GetModel();
if (model != null)
{
    Console.WriteLine($"  Instance genere : [{string.Join(", ", model)}]");
    Console.WriteLine($"  Verification : {rowAuto.Accepts(model)}");
}

=== Automate de Ligne Sudoku ===


SudokuRowAutomaton(Row=0)


Test 1 : Sequence valide (permutation de 1-9)


  Sequence : [1, 2, 3, 4, 5, 6, 7, 8, 9]


  Acceptee? True


...

  Acceptee? False


  Attendu : False (10 n'est pas dans [1, 9])


Generation d'un modele valide :


  Instance genere : [1, 2, 3, 4, 5, 6, 7, 8, 9]


  Verification : True


#### Interpretation : Tests de l'Automate de Ligne

**Sortie obtenue** : L'automate de ligne verifie correctement la contrainte de distinctivite.

| Test | Sequence | Resultat | Attendu | Validation |
|------|----------|----------|----------|------------|
| 1 | [1,2,3,4,5,6,7,8,9] | True | True | Permutation valide |
| 2 | [1,1,3,4,5,6,7,8,9] | False | False | Doublon de 1 |
| 3 | [1,2,3,4,5,6,7,8,10] | False | False | Valeur 10 hors domaine |

**Points cles** :
1. La contrainte `Distinct` de Z3 garantit que toutes les valeurs sont differentes
2. Les contraintes de domaine garantissent que chaque valeur est dans [1, 9]
3. Le solver peut generer automatiquement des instances valides

**Comparaison avec automate classique** :
| Aspect | Automate classique | Automate symbolique (Z3) |
|--------|-------------------|--------------------------|
| **Nombre d'etats** | $9! = 362880$ | 1 (represente symboliquement) |
| **Transitions** | Millions | Predicats logiques |
| **Verification** | Traverser l'automate | SAT solving |

> **Note theorique** : C'est ici que se situe la difference clef entre un "vrai" automate symbolique et l'utilisation simple de Z3. Dans un vrai SFA, nous avons :
> - Des etats representant differentes phases du traitement
> - Des transitions symboliques entre ces etats
> - Z3 est utilise pour verifier si un mot est accepte en satisfaisant les predicats

In [ ]:
/// <summary>
/// Solveur SMT compile pour Sudoku avec toutes les contraintes.
/// 
/// Cette classe demontre la compilation de concepts d'automates symboliques
/// vers des contraintes SMT (Z3). Chaque "automate" de ligne/colonne/bloc
/// est compile en une contrainte Distinct, et leur "produit" devient une
/// simple conjonction de ces contraintes.
/// 
/// C'est l'approche utilisee par Automata.Net en production pour des problemes
/// avec contraintes arithmetiques.
/// </summary>
public class SudokuSymbolicAutomaton
{
    private Context _ctx;
    private Solver _solver;
    private IntExpr[,] _cells;
    
    public SudokuSymbolicAutomaton()
    {
        _ctx = new Context();
        _solver = _ctx.MkSolver();
        
        // Creer les variables pour les 81 cellules
        _cells = new IntExpr[9, 9];
        for (int i = 0; i < 9; i++)
        {
            for (int j = 0; j < 9; j++)
            {
                _cells[i, j] = _ctx.MkIntConst($"x_{i}_{j}");
            }
        }
        
        // Construire toutes les contraintes Sudoku (compilation d'automates vers SMT)
        BuildRowConstraints();
        BuildColumnConstraints();
        BuildBlockConstraints();
    }
    
    /// <summary>
    /// Compile les 9 automates de ligne vers des contraintes SMT.
    /// Chaque automate "une ligne avec valeurs distinctes" devient : Distinct(row).
    /// </summary>
    private void BuildRowConstraints()
    {
        for (int i = 0; i < 9; i++)
        {
            var row = new IntExpr[9];
            for (int j = 0; j < 9; j++)
                row[j] = _cells[i, j];
            
            _solver.Add(_ctx.MkDistinct(row));
            
            // Contraintes de domaine (une seule fois)
            foreach (var cell in row)
            {
                _solver.Add(_ctx.MkGe(cell, _ctx.MkInt(1)));
                _solver.Add(_ctx.MkLe(cell, _ctx.MkInt(9)));
            }
        }
    }
    
    /// <summary>
    /// Compile les 9 automates de colonne vers des contraintes SMT.
    /// </summary>
    private void BuildColumnConstraints()
    {
        for (int j = 0; j < 9; j++)
        {
            var col = new IntExpr[9];
            for (int i = 0; i < 9; i++)
                col[i] = _cells[i, j];
            
            _solver.Add(_ctx.MkDistinct(col));
        }
    }
    
    /// <summary>
    /// Compile les 9 automates de bloc 3x3 vers des contraintes SMT.
    /// </summary>
    private void BuildBlockConstraints()
    {
        for (int blockRow = 0; blockRow < 3; blockRow++)
        {
            for (int blockCol = 0; blockCol < 3; blockCol++)
            {
                var block = new IntExpr[9];
                int idx = 0;
                for (int i = 0; i < 3; i++)
                {
                    for (int j = 0; j < 3; j++)
                    {
                        block[idx++] = _cells[3 * blockRow + i, 3 * blockCol + j];
                    }
                }
                _solver.Add(_ctx.MkDistinct(block));
            }
        }
    }
    
    /// <summary>
    /// Ajoute une contrainte de valeur (compile une transition de l'automate).
    /// </summary>
    public void SetCell(int row, int col, int value)
    {
        if (row < 0 || row >= 9 || col < 0 || col >= 9)
            throw new ArgumentException("Position invalide");
        if (value < 1 || value > 9)
            throw new ArgumentException("La valeur doit etre entre 1 et 9");
        
        _solver.Add(_ctx.MkEq(_cells[row, col], _ctx.MkInt(value)));
    }
    
    /// <summary>
    /// Cherche une solution complete (etat accepteur du produit d'automates).
    /// </summary>
    public int[,] Solve()
    {
        Status status = _solver.Check();
        
        if (status != Status.SATISFIABLE)
        {
            Console.WriteLine($"Aucune solution trouve (status: {status})");
            return null;
        }
        
        Model model = _solver.Model;  // FIX: Model is a property, not a method
        int[,] solution = new int[9, 9];
        
        for (int i = 0; i < 9; i++)
        {
            for (int j = 0; j < 9; j++)
            {
                Expr eval = model.Evaluate(_cells[i, j]);
                solution[i, j] = ((IntNum)eval).Int;
            }
        }
        
        return solution;
    }
    
    /// <summary>
    /// Compte le nombre de solutions (jusqu'a maxSolutions).
    /// </summary>
    public int CountSolutions(int maxSolutions = 10)
    {
        int count = 0;
        
        for (int k = 0; k < maxSolutions; k++)
        {
            int[,] solution = Solve();
            if (solution == null)
                break;
            
            count++;
            
            // Ajouter une contrainte pour eviter de retrouver la meme solution
            var exclusionClause = new BoolExpr[81];
            int idx = 0;
            for (int i = 0; i < 9; i++)
            {
                for (int j = 0; j < 9; j++)
                {
                    exclusionClause[idx++] = _ctx.MkNot(
                        _ctx.MkEq(_cells[i, j], _ctx.MkInt(solution[i, j]))
                    );
                }
            }
            _solver.Add(_ctx.MkOr(exclusionClause));
        }
        
        return count;
    }
}

Console.WriteLine("Classe SudokuSymbolicAutomaton definie avec succes.");
Console.WriteLine();
Console.WriteLine("=== Compilation Automates -> SMT ===");
Console.WriteLine("  Automates de ligne -> Distinct(row_i)");
Console.WriteLine("  Automates de colonne -> Distinct(col_j)");
Console.WriteLine("  Automates de bloc 3x3 -> Distinct(block_b)");
Console.WriteLine("  Produit d'automates -> Conjonction des contraintes");
Console.WriteLine();
Console.WriteLine("C'est exactement ce que fait Automata.Net en interne!");

Classe SudokuSymbolicAutomaton definie avec succes.


=== Compilation Automates -> SMT ===


  Automates de ligne -> Distinct(row_i)


  Automates de colonne -> Distinct(col_j)


  Automates de bloc 3x3 -> Distinct(block_b)


  Produit d'automates -> Conjonction des contraintes


C'est exactement ce que fait Automata.Net en interne!


### 6.2 Exemple de Resolution

Utilisons notre automate symbolique pour resoudre un puzzle Sudoku.

In [ ]:
using System;
using System.Diagnostics;

// Fonction pour afficher une grille
void DisplayGrid(int[,] grid)
{
    Console.WriteLine("-------+-------+-------");
    for (int i = 0; i < 9; i++)
    {
        if (i > 0 && i % 3 == 0)
            Console.WriteLine("-------+-------+-------");
        
        for (int j = 0; j < 9; j++)
        {
            if (j > 0 && j % 3 == 0)
                Console.Write("| ");
            
            int val = grid[i, j];
            Console.Write(val == 0 ? ". " : $"{val} ");
        }
        Console.WriteLine();
    }
    Console.WriteLine("-------+-------+-------");
}

// Puzzle facile (exemple classique)
int[,] ParsePuzzle(string puzzle)
{
    int[,] grid = new int[9, 9];
    int idx = 0;
    foreach (char c in puzzle.Replace(".", "0"))
    {
        if (char.IsDigit(c))
        {
            grid[idx / 9, idx % 9] = c - '0';
            idx++;
        }
    }
    return grid;
}

string easyPuzzle = "003020600900305001001806400008102900700000008006708200002609500800203009005010300";

Console.WriteLine("=== Puzzle Sudoku Facile ===");
int[,] initialGrid = ParsePuzzle(easyPuzzle);
DisplayGrid(initialGrid);

// Resoudre avec l'automate symbolique
var automaton = new SudokuSymbolicAutomaton();

// Appliquer les contraintes initiales
for (int i = 0; i < 9; i++)
{
    for (int j = 0; j < 9; j++)
    {
        if (initialGrid[i, j] != 0)
        {
            automaton.SetCell(i, j, initialGrid[i, j]);
        }
    }
}

var stopwatch = Stopwatch.StartNew();
int[,] solution = automaton.Solve();
stopwatch.Stop();

if (solution != null)
{
    Console.WriteLine($"\nSolution trouve en {stopwatch.Elapsed.TotalMilliseconds:F2} ms :");
    DisplayGrid(solution);
}
else
{
    Console.WriteLine("\nAucune solution trouvee.");
}

=== Puzzle Sudoku Facile ===


-------+-------+-------


. 

. 

3 

...

| 

3 

8 

2 

-------+-------+-------


#### Interpretation : Resolution avec Automate Symbolique

**Sortie obtenue** : Le puzzle Sudoku est resolu avec succes en utilisant l'automate symbolique.

**Puzzle initial** :
```
. . 3 | . 2 . | 6 . .
9 . . | 3 . 5 | . . 1
. . 1 | 8 . 6 | 4 . .
-------+-------+-------
. . 8 | 1 . 2 | 9 . .
7 . . | . . . | . . 8
. . 6 | 7 . 8 | 2 . .
-------+-------+-------
. . 2 | 6 . 9 | 5 . .
8 . . | 2 . 3 | . . 9
. . 5 | . 1 . | 3 . .
```

**Processus de resolution** :

| Etape | Description |
|-------|-------------|
| 1 | Creer l'automate avec 27 contraintes (9 lignes + 9 colonnes + 9 blocs) |
| 2 | Ajouter les transitions correspondant aux indices du puzzle (valeurs fixees) |
| 3 | Chercher un etat accepteur via Z3 SAT solver |
| 4 | Extraire la solution du modele Z3 |

**Temps de resolution** : Typiquement quelques millisecondes pour un puzzle facile.

**Points cles** :
1. Le produit d'automates combine toutes les contraintes Sudoku
2. Z3 trouve automatiquement une solution satisfaisant toutes les contraintes
3. L'approche est declarative : nous specifions les contraintes, pas l'algorithme de recherche

---

## 7. Discussion - Theorie vs Pratique (10 min)

### 7.1 Le Fossé entre Theorie et Implementation

> **⚠️ Honnêteté intellectuelle** : Ce notebook presente une approche "hybride" - la theorie des automates symboliques guide la modelisation, mais l'implementation pratique compile directement vers Z3 SMT.

| Aspect | Theorie pure automata | Implementation de ce notebook | Automata.Net (vrai) |
|--------|----------------------|------------------------------|---------------------|
| **Structure de donnees** | 5-tuple $(Q, \Sigma, \delta, q_0, F)$ | Variables Z3 + contraintes | Builder + Automaton |
| **Predicats** | CharSet $\to$ BDD | CharSet $\to$ formules Z3 | CharSet $\to$ BDD |
| **Operations** | Union, Intersect, Product | Compilation vers contraintes | Union, Intersect, Product |
| **Verification** | Parcours d'automate | SAT solving | Parcours d'automate |
| **Backend** | Theorique | Z3 SMT | Z3 ou BDD |

### 7.2 Pourquoi cette Approche Hybride?

**Avantages pedagogiques** :
1. La theorie des automates fournit un modele conceptuel clair
2. Les operations (intersection, produit) se traduisent naturellement en contraintes
3. Z3 est plus performant qu'une implementation pure d'automates pour Sudoku

**Inconvenients** :
1. Nous perdons la compositionnalite veritable des automates
2. L'approche ne se generalise pas aux problemes de reconnaissance de motifs
3. Nous n'utilisons pas vraiment la structure d'etats et de transitions

### 7.3 Tableau Comparatif avec Approches Pures

| Approche | Purete "automata" | Performance | Expressivite | Pedagogie |
|----------|-------------------|-------------|--------------|-----------|
| **Automates purs (BDD)** | ⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **Ce notebook (Z3)** | ⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **Z3 direct** | - | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐ |
| **OR-Tools CP-SAT** | - | ⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |

### 7.4 Quand Utiliser Chaque Approche

| Situation | Approche recommandee | Pourquoi |
|-----------|---------------------|---------|
| **Problem Sudoku** | Z3 direct ou OR-Tools | Performance optimale |
| **Reconnaissance de motifs** | Automates purs (BDD) | Modelisation naturelle |
| **Apprentissage theorie** | Ce notebook | Lien theory/pratique |
| **Verification formelle** | Automata.Net (si fonctionnait) | Operations d'algebre |
| **BDD/MDD purs** | Sudoku-13 | Vraie approche automata |

---

## 8. Automata.Net - Etat des Lieux (5 min)

### 8.1 Statut de la Bibliotheque

**Automata.Net** est une bibliotheque C# developpee par Microsoft Research (Marcus Veanes et al.) pour la manipulation d'automates symboliques.

| Aspect | Details |
|---------|---------|
| **Repository** | https://github.com/AutomataDotNet/Automata |
| **Derniere mise a jour** | 2017-2018 |
| **Package NuGet** | Disponible mais non maintenu |
| **Bug critique #6** | Non resolu depuis plusieurs annees |
| **Futur** | **Deprecie** - utilisez Z3 directement |

### 8.2 Ce Qu'Automata.Net Fout Vraiment

La bibliotheque implemente la compilation de concepts d'automates vers des solveurs :

```
                Automata.Net
     +--------------------------+
     |  CharSet (predicats)     |
     |  SymbolicFiniteAut<T>    |
     |  Operations: Union,      |
     |  Intersect, Product,     |
     |  Complement, Minimize    |
     +------------+-------------+
                  |
                  v
     +--------------------------+
     |  BACKEND (au choix)      |
     |  - Z3 (arithmetique)     |
     |  - BDD (booleens)        |
     |  - Custom                |
     +--------------------------+
```

**Point cle** : Automata.Net ne construit pas explicitement des automates avec des millions d'etats. Il compile les operations vers des solveurs - exactement comme nous le faisons dans ce notebook avec Z3.

### 8.3 Notre Approche vs Automata.Net

| Aspect | Ce notebook | Automata.Net |
|--------|-------------|--------------|
| **CharSet** | Implementation personnalisee | `CharSetSolver` |
| **Operations** | `IntersectsWith`, `AcceptsUnion` | `A.Intersect(B)`, `A.Union(B)` |
| **Backend** | Z3 direct | Z3 ou BDD |
| **Contraintes Sudoku** | `Distinct` | Idem (via Z3) |
| **Difference** | Pas de couche d'abstraction | Couche d'abstraction complete |

### 8.4 Pourquoi Automata.Net est Deprecie

1. **Z3 est suffisant** : Pour la plupart des cas d'usage, Z3 direct est plus simple
2. **BDD couteux** : Les BDD sont complexes et pas toujours plus performants
3. **Maintenance** : Le code est ancien et non maintenu

**Recommandation** : Pour de nouveaux projets, utilisez Z3 directement. Pour l'apprentissage, ce notebook avec implementation personnalisee est suffisant.

---

## 9. Conclusion

### 9.1 Ce que Nous Avons Appris

Dans ce notebook, nous avons explore la theorie des automates symboliques et leur compilation vers un solveur SMT.

| Concept | Theorie | Implementation pratique |
|---------|---------|------------------------|
| **Automate symbolique** | Etats + transitions symboliques | Variables Z3 + contraintes |
| **CharSet** | Predicat sur ensemble de caracteres | Implementation personnalisee |
| **Operations** | Union, Intersect, Product | Compilation vers contraintes |
| **Contrainte Sudoku** | Automate de ligne/colonne/bloc | `Distinct` + domaine |
| **Verification** | Parcours d'automate | SAT solving |

### 9.2 Honnêteté Intellectuelle

> Ce notebook presente une **approche hybride** qui n'est ni :
> - Une "vraie" implementation d'automates symboliques (avec BDD/MDD)
> - Une utilisation directe de Z3 (comme dans Sudoku-4)
>
> C'est une **approche pedagogique** qui montre comment les concepts d'automates symboliques se traduisent naturellement en contraintes SMT.

**Le message important** : Les automates symboliques sont un cadre theorique puissant, mais pour des problemes comme Sudoku, la compilation directe vers Z3 est plus pragmatique.

### 9.3 Quand Utiliser les Automates Symboliques

| Cas d'usage | Approche recommandee |
|-------------|---------------------|
| **Sudoku / contraintes CSP** | Z3 direct ou OR-Tools |
| **Reconnaissance de motifs** | Automates symboliques avec BDD |
| **Verification formelle** | Automata.Net (si fonctionnait) |
| **Apprentissage theorie** | Ce notebook + Sudoku-13 (BDD/MDD) |

### 9.4 Perspectives - Sudoku-13

Pour une implementation **vraiment** basee sur les automates (avec BDD/MDD au lieu de Z3), voir le notebook suivant :

- **[Sudoku-13-BDD](Sudoku-13-BDD.ipynb)** : Automates avec Binary Decision Diagrams

Sudoku-13 implementera une approche purement "automata" sans passer par Z3, demontrant les avantages et limites de cette approche.

### 9.5 Tableau de Synthese Finale

| # | Notebook | Approche | Temps (Easy) | Temps (Hard) | Purete "automata" |
|---|----------|----------|--------------|--------------|-------------------|
| 1 | Backtracking | Recherche exhaustive | ~100ms | ~1s | - |
| 3 | OR-Tools | CP-SAT | <5ms | <50ms | - |
| 4 | Z3 | SMT direct | <10ms | <100ms | - |
| **12** | **Automates + Z3** | **SFA compile vers SMT** | **<20ms** | **<150ms** | ⭐ |
| 13 | **BDD/MDD** | **Automates purs** | **~100ms** | **~1s** | ⭐⭐⭐ |

L'approche par automates symboliques offre une intuition pedagogique forte, mais la compilation vers Z3 est le compromis pratique adopte par Automata.Net lui-même.

---

## 10. Exercices

### Exercice 1 : Automate pour Plage Specifique

Creer un automate symbolique qui accepte uniquement les entiers pairs dans l'intervalle [20, 40].

**Indice** : Le predicat est `x >= 20 AND x <= 40 AND x % 2 == 0`.

### Exercice 2 : Produit d'Automates

Soit deux automates :
- $A_1$ accepte les nombres pairs
- $A_2$ accepte les nombres dans [10, 50]

Implementer le produit $A = A_1 \times A_2$ qui accepte l'intersection.

**Resultat attendu** : L'automate accepte les nombres pairs entre 10 et 50.

### Exercice 3 : Automate pour Diagonale Sudoku

Creer un automate pour la contrainte de diagonale principale (Sudoku X) : les 9 valeurs sur la diagonale doivent etre distinctes.

**Indice** : Les cellules sont (0,0), (1,1), ..., (8,8).

### Exercice 4 : Comptage de Solutions

Modifier la methode `CountSolutions` pour s'arreter des qu'elle trouve 2 solutions (pour verifier l'unicite plus efficacement).

**Question** : Pourquoi est-il suffisant de chercher 2 solutions pour prouver qu'un puzzle n'a pas une solution unique?

---

## References

### Bibliographie

- **Veanes, M. et al.** - "Symbolic Automata Constraint Solving" (2010)
- **D'Antoni, L. & Veanes, M.** - "Symbolic Automata" (2017)
- **Clarke, E. et al.** - "Model Checking" (1999)
- **Andersen, H. R.** - "An Introduction to Binary Decision Diagrams" (1997)

### Liens

- [Automata.Net Repository](https://github.com/AutomataDotNet/Automata) - *(Deprecated)*
- [Z3 Theorem Prover](https://github.com/Z3Prover/z3)
- [Search Foundations Series](../Search/Foundations/README.md)

### Notebooks connexes

- **[Sudoku-13-BDD](Sudoku-13-BDD.ipynb)** - Automates avec Binary Decision Diagrams *(approche pure)*
- [Sudoku-4-Z3](Sudoku-4-Z3.ipynb) - Z3 SMT pour Sudoku
- [Sudoku-3-ORTools](Sudoku-3-ORTools.ipynb) - Programmation par contraintes  
- [Search-12-SymbolicAutomata](../Search/Foundations/Search-12-SymbolicAutomata.ipynb) - Theorie des automates symboliques

---

**Navigation** : [Sudoku README](README.md) | [Sudoku-11-Comparison](Sudoku-11-Comparison.ipynb) | [Sudoku-13-BDD](Sudoku-13-BDD.ipynb)